In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score

In [2]:
df = pd.read_csv("/Users/valazeinali/Desktop/Dartmouth/Spring 2021/2017-2018")
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,GID,Last,First,First Last,Date,Team,Opp,H/A,GameID,GTime(ET),...,DK Change,DD Sal,DD Change,YH Sal,YH Change,FD pos,DK pos,DD pos,YH pos,Unnamed: 35
0,3816,Williams,Louis,Louis Williams,20180411.0,lac,lal,H,20180411-lal-lac,22.5,...,-200.0,NaN,NaN,27.0,0.0,2.0,12.0,NaN,2.0,NaN
1,3816,Williams,Louis,Louis Williams,20180409.0,lac,nor,H,20180409-nor-lac,22.5,...,-1900.0,NaN,NaN,27.0,0.0,2.0,12.0,NaN,2.0,NaN
2,3816,Williams,Louis,Louis Williams,20180407.0,lac,den,H,20180407-den-lac,15.5,...,2200.0,NaN,NaN,27.0,-1.0,2.0,12.0,NaN,2.0,NaN
3,3816,Williams,Louis,Louis Williams,20180405.0,lac,uta,A,20180405-lac-uta,21.0,...,-800.0,NaN,NaN,28.0,-1.0,2.0,12.0,NaN,2.0,NaN
4,3816,Williams,Louis,Louis Williams,20180403.0,lac,sas,H,20180403-sas-lac,22.5,...,-200.0,NaN,NaN,29.0,0.0,2.0,12.0,NaN,2.0,NaN


In [3]:
df = df[["GID","First  Last", "Date", "Team pts", "Minutes", "FDP", "DKP", "FD Sal", "DK Sal"]]
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal
0,3816,Louis Williams,20180411.0,100.0,0.000000,0.0,0.00,6700.0,6700.0
1,3816,Louis Williams,20180409.0,100.0,0.000000,0.0,0.00,6700.0,6900.0
2,3816,Louis Williams,20180407.0,115.0,31.483333,29.5,31.50,6700.0,8800.0
3,3816,Louis Williams,20180405.0,95.0,30.466667,13.7,15.75,6600.0,6600.0
4,3816,Louis Williams,20180403.0,113.0,36.866667,32.9,32.50,6700.0,7400.0


In [4]:
df = df.reindex(index=df.index[::-1]) # reverse the order of the df to create appropriate prediction model
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal
43153,A comparable file for the 2015-16 NBA season i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43152,A comparable file for the 2016-17 NBA season i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43151,A data legend and other notes about this file ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43150,4817,Willie Cauley-Stein,20171018.0,100.0,31.633333,42.5,42.00,4700.0,5900.0
43149,4817,Willie Cauley-Stein,20171020.0,93.0,26.850000,27.7,28.75,6300.0,6200.0


In [5]:
df = df[df["First  Last"].notna()] # get rid of NA players
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal
43150,4817,Willie Cauley-Stein,20171018.0,100.0,31.633333,42.5,42.00,4700.0,5900.0
43149,4817,Willie Cauley-Stein,20171020.0,93.0,26.850000,27.7,28.75,6300.0,6200.0
43148,4817,Willie Cauley-Stein,20171021.0,79.0,26.533333,13.1,13.50,6300.0,6200.0
43147,4817,Willie Cauley-Stein,20171023.0,115.0,28.166667,22.8,23.00,5800.0,6000.0
43146,4817,Willie Cauley-Stein,20171026.0,106.0,29.700000,27.9,25.00,5700.0,5600.0


In [6]:
df = df.drop(df[df.Minutes < 1].index) # drop games that players dont play becasue it will mess up RSI calculation
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal
43150,4817,Willie Cauley-Stein,20171018.0,100.0,31.633333,42.5,42.00,4700.0,5900.0
43149,4817,Willie Cauley-Stein,20171020.0,93.0,26.850000,27.7,28.75,6300.0,6200.0
43148,4817,Willie Cauley-Stein,20171021.0,79.0,26.533333,13.1,13.50,6300.0,6200.0
43147,4817,Willie Cauley-Stein,20171023.0,115.0,28.166667,22.8,23.00,5800.0,6000.0
43146,4817,Willie Cauley-Stein,20171026.0,106.0,29.700000,27.9,25.00,5700.0,5600.0


In [7]:
df["Salary/Point FDP"] = df["FD Sal"] / df["FDP"] # Fan Duel salary per point
df["Salary/Point DKP"] = df["DK Sal"] / df["DKP"] # draft kings salary per point
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal,Salary/Point FDP,Salary/Point DKP
43150,4817,Willie Cauley-Stein,20171018.0,100.0,31.633333,42.5,42.00,4700.0,5900.0,110.588235,140.476190
43149,4817,Willie Cauley-Stein,20171020.0,93.0,26.850000,27.7,28.75,6300.0,6200.0,227.436823,215.652174
43148,4817,Willie Cauley-Stein,20171021.0,79.0,26.533333,13.1,13.50,6300.0,6200.0,480.916031,459.259259
43147,4817,Willie Cauley-Stein,20171023.0,115.0,28.166667,22.8,23.00,5800.0,6000.0,254.385965,260.869565
43146,4817,Willie Cauley-Stein,20171026.0,106.0,29.700000,27.9,25.00,5700.0,5600.0,204.301075,224.000000


In [8]:
# RSI momnetum 
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [9]:
# my_list = []
# for i in df["GID"]:
#     #print(i)
#     my_list.append(i)
    
# numbers = [ int(x) for x in my_list] # convert everyting to an int
# print(numbers)

In [10]:
df["GID"] = [int(x) for x in df["GID"]] # convert everyting to an int
df.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal,Salary/Point FDP,Salary/Point DKP
43150,4817,Willie Cauley-Stein,20171018.0,100.0,31.633333,42.5,42.00,4700.0,5900.0,110.588235,140.476190
43149,4817,Willie Cauley-Stein,20171020.0,93.0,26.850000,27.7,28.75,6300.0,6200.0,227.436823,215.652174
43148,4817,Willie Cauley-Stein,20171021.0,79.0,26.533333,13.1,13.50,6300.0,6200.0,480.916031,459.259259
43147,4817,Willie Cauley-Stein,20171023.0,115.0,28.166667,22.8,23.00,5800.0,6000.0,254.385965,260.869565
43146,4817,Willie Cauley-Stein,20171026.0,106.0,29.700000,27.9,25.00,5700.0,5600.0,204.301075,224.000000


In [11]:
df1 = df[df["GID"] == 4379] # Try to do momentum strat for Kyrie as an example
df1.head()

,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal,Salary/Point FDP,Salary/Point DKP
346,4379,Kyrie Irving,20171017.0,99.0,39.350000,48.8,50.50,8200.0,8300.0,168.032787,164.356436
345,4379,Kyrie Irving,20171018.0,100.0,38.933333,32.3,32.00,8200.0,8500.0,253.869969,265.625000
344,4379,Kyrie Irving,20171020.0,102.0,35.800000,35.2,36.50,9000.0,8600.0,255.681818,235.616438
343,4379,Kyrie Irving,20171024.0,110.0,29.533333,34.1,35.25,9100.0,8700.0,266.862170,246.808511
342,4379,Kyrie Irving,20171026.0,96.0,34.583333,45.1,44.25,8900.0,8700.0,197.339246,196.610169


In [12]:
df1['6D_RSI'] = computeRSI(df1['Salary/Point FDP'], 5) 
df1['6D_RSI'] = computeRSI(df1['Salary/Point DKP'], 5) 
df1.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal,Salary/Point FDP,Salary/Point DKP,6D_RSI
346,4379,Kyrie Irving,20171017.0,99.0,39.350000,48.8,50.50,8200.0,8300.0,168.032787,164.356436,NaN
345,4379,Kyrie Irving,20171018.0,100.0,38.933333,32.3,32.00,8200.0,8500.0,253.869969,265.625000,NaN
344,4379,Kyrie Irving,20171020.0,102.0,35.800000,35.2,36.50,9000.0,8600.0,255.681818,235.616438,NaN
343,4379,Kyrie Irving,20171024.0,110.0,29.533333,34.1,35.25,9100.0,8700.0,266.862170,246.808511,NaN
342,4379,Kyrie Irving,20171026.0,96.0,34.583333,45.1,44.25,8900.0,8700.0,197.339246,196.610169,NaN
341,4379,Kyrie Irving,20171028.0,96.0,33.550000,43.3,41.50,8900.0,8400.0,205.542725,202.409639,49.508454
340,4379,Kyrie Irving,20171030.0,108.0,30.333333,46.4,44.50,8200.0,8000.0,176.724138,179.775281,39.377105
339,4379,Kyrie Irving,20171101.0,113.0,27.000000,39.5,38.50,8400.0,8500.0,212.658228,220.779221,58.573920
338,4379,Kyrie Irving,20171103.0,101.0,34.483333,49.0,47.75,8400.0,8400.0,171.428571,175.916230,40.872834
337,4379,Kyrie Irving,20171105.0,104.0,27.666667,20.5,20.00,8400.0,8500.0,409.756098,425.000000,80.910162


In [13]:
df1["FDP Change"] = df1["FDP"].pct_change() * 100;
df1["DKP Change"] = df1["DKP"].pct_change() * 100;
df1.head(20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GID,First Last,Date,Team pts,Minutes,FDP,DKP,FD Sal,DK Sal,Salary/Point FDP,Salary/Point DKP,6D_RSI,FDP Change,DKP Change
346,4379,Kyrie Irving,20171017.0,99.0,39.350000,48.8,50.50,8200.0,8300.0,168.032787,164.356436,NaN,NaN,NaN
345,4379,Kyrie Irving,20171018.0,100.0,38.933333,32.3,32.00,8200.0,8500.0,253.869969,265.625000,NaN,-33.811475,-36.633663
344,4379,Kyrie Irving,20171020.0,102.0,35.800000,35.2,36.50,9000.0,8600.0,255.681818,235.616438,NaN,8.978328,14.062500
343,4379,Kyrie Irving,20171024.0,110.0,29.533333,34.1,35.25,9100.0,8700.0,266.862170,246.808511,NaN,-3.125000,-3.424658
342,4379,Kyrie Irving,20171026.0,96.0,34.583333,45.1,44.25,8900.0,8700.0,197.339246,196.610169,NaN,32.258065,25.531915
341,4379,Kyrie Irving,20171028.0,96.0,33.550000,43.3,41.50,8900.0,8400.0,205.542725,202.409639,49.508454,-3.991131,-6.214689
340,4379,Kyrie Irving,20171030.0,108.0,30.333333,46.4,44.50,8200.0,8000.0,176.724138,179.775281,39.377105,7.159353,7.228916
339,4379,Kyrie Irving,20171101.0,113.0,27.000000,39.5,38.50,8400.0,8500.0,212.658228,220.779221,58.573920,-14.870690,-13.483146
338,4379,Kyrie Irving,20171103.0,101.0,34.483333,49.0,47.75,8400.0,8400.0,171.428571,175.916230,40.872834,24.050633,24.025974
337,4379,Kyrie Irving,20171105.0,104.0,27.666667,20.5,20.00,8400.0,8500.0,409.756098,425.000000,80.910162,-58.163265,-58.115183


In [14]:
final_result = df1[["First  Last", "Date", "FDP", "DKP", "6D_RSI", "DKP Change", "FDP Change"]]
final_result['Signal'] = np.where(final_result['6D_RSI'] >= 50, 1, 0) # is RSI >= to 50? If yes, buy for tomorrows game

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
final_result['Bought'] = final_result.Signal.shift(1)
    
# Fill the NaN that is introduced into the 'Bought' column by the shift
final_result['Bought'] = final_result['Bought'].fillna(0)
    
final_result.head(20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,First Last,Date,FDP,DKP,6D_RSI,DKP Change,FDP Change,Signal,Bought
346,Kyrie Irving,20171017.0,48.8,50.50,NaN,NaN,NaN,0,0.0
345,Kyrie Irving,20171018.0,32.3,32.00,NaN,-36.633663,-33.811475,0,0.0
344,Kyrie Irving,20171020.0,35.2,36.50,NaN,14.062500,8.978328,0,0.0
343,Kyrie Irving,20171024.0,34.1,35.25,NaN,-3.424658,-3.125000,0,0.0
342,Kyrie Irving,20171026.0,45.1,44.25,NaN,25.531915,32.258065,0,0.0
341,Kyrie Irving,20171028.0,43.3,41.50,49.508454,-6.214689,-3.991131,0,0.0
340,Kyrie Irving,20171030.0,46.4,44.50,39.377105,7.228916,7.159353,0,0.0
339,Kyrie Irving,20171101.0,39.5,38.50,58.573920,-13.483146,-14.870690,1,0.0
338,Kyrie Irving,20171103.0,49.0,47.75,40.872834,24.025974,24.050633,0,1.0
337,Kyrie Irving,20171105.0,20.5,20.00,80.910162,-58.115183,-58.163265,1,0.0


In [16]:
final_result = final_result[["FDP", "DKP", "6D_RSI", "DKP Change", "FDP Change","Bought"]]
final_result = final_result.replace([np.inf, -np.inf], np.nan)
final_result.dropna(inplace=True)
y = final_result["Bought"]
X = final_result.drop(columns=["Bought"])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


# Decision Tree

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
yPred = clf.predict(X_test)
accuracy_score(y_test, yPred)

0.8181818181818182

# Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression()
logReg.fit(X_train, y_train)
yPred = logReg.predict(X_test)
accuracy_score(y_test, yPred)

0.9090909090909091

In [20]:
y_pred = logReg.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,np.around(yPred,0)).ravel()
print('Precision:',tp/(tp+fp))
print('Recall:',tp/(tp+fn))
print('True Positive Rate:',tp/(tp+fn))
print('False Positive Rate:',fp/(fp+tn))
conf_matrix = confusion_matrix(y_test,np.around(yPred,0))
print(conf_matrix)

print('Accuracy:',accuracy_score(y_test,np.around(yPred,0)))

Precision: 1.0
Recall: 0.75
True Positive Rate: 0.75
False Positive Rate: 0.0
[[7 0]
 [1 3]]
Accuracy: 0.9090909090909091


# gradient boosts

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

gBoostClassifier = GradientBoostingClassifier()
gBoostClassifier.fit(X_train, y_train)
yPred = gBoostClassifier.predict(X_test)
accuracy_score(y_test, yPred)

0.7272727272727273

In [22]:
y_pred = gBoostClassifier.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,np.around(yPred,0)).ravel()
print('Precision:',tp/(tp+fp))
print('Recall:',tp/(tp+fn))
print('True Positive Rate:',tp/(tp+fn))
print('False Positive Rate:',fp/(fp+tn))
conf_matrix = confusion_matrix(y_test,np.around(yPred,0))
print(conf_matrix)

print('Accuracy:',accuracy_score(y_test,np.around(yPred,0)))

Precision: 0.6
Recall: 0.75
True Positive Rate: 0.75
False Positive Rate: 0.2857142857142857
[[5 2]
 [1 3]]
Accuracy: 0.7272727272727273


# adaboost

In [23]:
from sklearn.ensemble import AdaBoostClassifier

adaBoostClassifier = AdaBoostClassifier()
adaBoostClassifier.fit(X_train, y_train)
yPred = adaBoostClassifier.predict(X_test)
accuracy_score(y_test, yPred)

0.7272727272727273

In [24]:
y_pred = adaBoostClassifier.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,np.around(yPred,0)).ravel()
print('Precision:',tp/(tp+fp))
print('Recall:',tp/(tp+fn))
print('True Positive Rate:',tp/(tp+fn))
print('False Positive Rate:',fp/(fp+tn))
conf_matrix = confusion_matrix(y_test,np.around(yPred,0))
print(conf_matrix)

print('Accuracy:',accuracy_score(y_test,np.around(yPred,0)))

Precision: 0.6
Recall: 0.75
True Positive Rate: 0.75
False Positive Rate: 0.2857142857142857
[[5 2]
 [1 3]]
Accuracy: 0.7272727272727273


# Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
yPred = rf.predict(X_test)
accuracy_score(y_test, yPred)

0.8181818181818182

In [26]:
y_pred = rf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,np.around(yPred,0)).ravel()
print('Precision:',tp/(tp+fp))
print('Recall:',tp/(tp+fn))
print('True Positive Rate:',tp/(tp+fn))
print('False Positive Rate:',fp/(fp+tn))
conf_matrix = confusion_matrix(y_test,np.around(yPred,0))
print(conf_matrix)

print('Accuracy:',accuracy_score(y_test,np.around(yPred,0)))

Precision: 0.75
Recall: 0.75
True Positive Rate: 0.75
False Positive Rate: 0.14285714285714285
[[6 1]
 [1 3]]
Accuracy: 0.8181818181818182


In [27]:
# i = 0
# for ID in enumerate(numbers):
#     #df[i] = pd.DataFrame()
#     #vars()[ID] = pd.DataFrame()
#     df = df[df["GID"] == ID] # create a dataframe for each player
#     #df.head()
#     #i+=1 # increment

In [28]:
#df['preTestScore'].groupby(df['regiment']).describe()